# Camera-Basic-Recognition_static

O Sistema deve identificar as provas a serem analisadas, independente de sua posição

#### [1]Desta forma haverá um delimitador na câmera, onde analizará apenas as informações contidas em seu espaço

#### [2]Desta forma após identificar o padrão, irá contabilizar 3 segundos para salvar uma cópia

Identificação de alternativas

In [1]:
def identify_alternative(image,cropped):
    try:
        global init
        gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 5)
        
        rows = gray.shape[0]
        circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1,rows/10,param1=50,param2=30,minRadius=0, maxRadius=30)

        if circles is not None:
            circles = np.uint16(np.around(circles))
            
            for i in circles[0, :]:
                center = (i[0], i[1])
                
                #Ponto central
                cv2.circle(image, center, 1, BLUE, 3)
                
                #Circulo externo
                radius = i[2]
                cv2.circle(image, center, radius, GREEN, 3)
            end = time()
            cv2.imshow("circles", image)
            if(init and int(end - init) >= 3):
                return True
        else:
            cv2.imshow("detected circles", image)
            init = time()
    except:
        cv2.destroyAllWindows()
        exit(0)

Detecção de Borda(Retângulo)

In [2]:
def delimiter(image,color):
    height, width = image.shape[:2]
    # Dimensões relativas da tela
    top_left_x = int(width / 8)
    top_left_y = int((height / 2) + (height / 3))
    bottom_right_x = int((width / 2.3) * 2)
    bottom_right_y = int((height / 2) - (height / 3))
    # Desenhando retangulo para delimitar o que será analisado
    cv2.rectangle(image, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), color, 2)
   
    mask = np.zeros(image.shape[:2], dtype = "uint8")
    cropped = image[bottom_right_y:top_left_y , top_left_x:bottom_right_x]  
    cropped_mask = cv2.bitwise_and(image.copy(), image.copy(), mask = mask)
    cropped_mask[bottom_right_y:top_left_y , top_left_x:bottom_right_x] = cropped
    
    return cropped_mask,cropped

In [3]:
def identify_test(image):  
    cropped_mask,_ = delimiter(image,color)
    
    status = identify_alternative(image,cropped_mask)  
    _, crooped = delimiter(image,color)
    return status,crooped

In [4]:
def save_copy(image):
    name = "dataset/imgs/identificadas/{}.png".format(str(datetime.now()))
    cv2.imwrite(name,image)            

In [5]:
def camera_processing():
    
    cap = cv2.VideoCapture(0)

    while(True):        
        ret, frame = cap.read()
        frame = cv2.flip(frame,1)
        status,cropped = identify_test(frame)
        if (cv2.waitKey(1) & 0xFF == ord('q')) or status: 
            save_copy(cropped)
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [6]:
from datetime import datetime
from time import time
import numpy as np
import imutils
import sys
import cv2
color = (0,255,0)
BLUE = (255,0,0)
GREEN = (0,255,0)
camera_processing()

cv2.waitKey(0)
cv2.destroyAllWindows()